<a href="https://colab.research.google.com/github/GuillermoCarbajal/Blur2Seq/blob/master/Blur2Seq_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Blur2Seq demo

## Clone the repo

In [ ]:
!git clone https://github.com/GuillermoCarbajal/Blur2Seq.git
%cd Blur2Seq

## Setup the environment

In [ ]:
!pip install -r 'requirements.txt'

## Download pretrained models

In [ ]:
import gdown
gdown.download('http://iie.fing.edu.uy/~carbajal/J-MTPD/camera_shake_epoch25_epoch35_epoch50_epoch10_epoch5_epoch25_epoch25_epoch25_epoch27_epoch24_epoch4_epoch10_epoch22_epoch23_epoch90.pkl', "./pretrained_models/", quiet=False)
gdown.download('http://iie.fing.edu.uy/~carbajal/J-MTPD/camera_shake_epoch25_epoch35_epoch50_epoch10_epoch5_epoch25_epoch25_epoch25_epoch27_epoch24_epoch4_epoch10_epoch22_epoch23_epoch90_G.pkl', "./pretrained_models/", quiet=False)

In [ ]:
import torch
from skimage.io import imread, imsave
from skimage import img_as_ubyte
import os
from models.CameraShakeModelTwoBranches import CameraShakeModelTwoBranches as CameraShakeModel
from models.network_nimbusr_offsets import NIMBUSR_Offsets as net_nimbusr_offsets
from torchvision import transforms
import numpy as np
from torchvision.utils import make_grid
from utils.homographies import compute_intrinsics, get_offsets_from_positions, generate_video, show_kernels_from_offsets_on_blurry_image
from utils.visualization import save_image, tensor2im, show_positions_found, sort_positions
from matplotlib import pyplot  as plt
from skimage.color import gray2rgb
from skimage.transform import resize

## Trajectory Computation

In [ ]:
#input_image = 'testing_imgs/0316.png'
input_image ='testing_imgs/Blurry2_1.png'
camera_model_file = 'pretrained_models/camera_shake_epoch25_epoch35_epoch50_epoch10_epoch5_epoch25_epoch25_epoch25_epoch27_epoch24_epoch4_epoch10_epoch22_epoch23_epoch90.pkl'

K=25
camera_model = CameraShakeModel(K).cuda()
camera_model.load_state_dict( torch.load(camera_model_file), strict=False)
camera_model.eval()

In [ ]:
def compute_trajectory(input_file, reblur_model, output_dir='testing_results', focal_length=-1,
                       gamma_factor=1.0, save_video=False):

  n_positions= 25 # number of positions estimated by the network
  if not os.path.exists(output_dir):
      os.makedirs(output_dir)

  img_name = input_file.split('/')[-1]
  img_name, ext = img_name.split('.')

  print('loading image ',input_file)
  blurry_image = imread(input_file)/255.0

  # Blurry image is transformed to pytorch format
  blurry_tensor = torch.from_numpy(blurry_image).permute(2,0,1)[None].cuda().float()

  _, C,H,W = blurry_tensor.shape

  # Kernels and masks are estimated
  blurry_tensor_to_compute_kernels = blurry_tensor**gamma_factor - 0.5


  if focal_length > 0:
      f = torch.Tensor([focal_length]).to(blurry_tensor.device)
      #f = torch.Tensor([float(max(H,W))]).to(tensor_img.device)
      intrinsics = torch.Tensor([[f, 0, W/2],[0, f, H/2], [0, 0, 1] ]).cuda(blurry_tensor.device)
      intrinsics = intrinsics[None,:,:]
  else:
      intrinsics = compute_intrinsics(W,H).cuda(blurry_tensor.device)[None]
      f =  torch.Tensor([max(H,W)]).to(blurry_tensor.device)

      #focal_channel = f/maximo * torch.ones(N,1,H,W).to(tensor_img.device)
      #cam_input = torch.concat((focal_channel, tensor_img), dim=1)

  # get positions
  with torch.no_grad():
      camera_positions = reblur_model(blurry_tensor_to_compute_kernels,f)

  order = sort_positions(camera_positions[0])
  camera_positions[0] = camera_positions[0,order,:]

  offsets = get_offsets_from_positions(blurry_tensor.shape, camera_positions, intrinsics)
  offsets = offsets.reshape(1,2*n_positions, H,W)
  offsets_BT=None



  found_positions_np = camera_positions[0].detach().cpu().numpy()
  np.savetxt(os.path.join(output_dir,f'{img_name}_found_positions.txt'), found_positions_np)
  pose = np.zeros((found_positions_np.shape[0], 6))
  pose[:, 3:] = found_positions_np

  #K, _ = generarK((H,W,C), pose, A=intrinsics[0].detach().cpu().numpy())
  kernels_file = os.path.join(output_dir, img_name + '_kernels_found.png' )
  #kernels_estimated = mostrar_kernels(K, (H,W,C), output_name = kernels_file)


  show_kernels_from_offsets_on_blurry_image(blurry_tensor[0],offsets[0].reshape(n_positions,2,H,W), kernels_file)
  print('Kernels saved in ',os.path.join(output_dir, img_name + '_kernels.png') )



In [ ]:
compute_trajectory(input_image, camera_model)

## Image Deblurring

In [ ]:
#from models.network_nimbusr_pmbm import NIMBUSR_PMBM as net
from models.network_nimbusr_offsets import NIMBUSR_Offsets as net_nimbusr_offsets

In [ ]:
def load_nimbusr_net(restoration_network_file, type='offsets'):
    opt_net = { "n_iter": 8
        , "h_nc": 64
        , "in_nc": 4 #2 if args.gray else 4 #4
        , "out_nc":3 #1 if args.gray else 3 #3
        #, "ksize": 25
        , "nc": [64, 128, 256, 512]
        , "nb": 2
        , "gc": 32
        , "ng": 2
        , "reduction" : 16
        , "act_mode": "R"
        , "upsample_mode": "convtranspose"
        , "downsample_mode": "strideconv"}


    if type=='pmbm':
        netG = net(n_iter=opt_net['n_iter'],
                    h_nc=opt_net['h_nc'],
                    in_nc=opt_net['in_nc'],
                    out_nc=opt_net['out_nc'],
                    nc=opt_net['nc'],
                    nb=opt_net['nb'],
                    act_mode=opt_net['act_mode'],
                    downsample_mode=opt_net['downsample_mode'],
                    upsample_mode=opt_net['upsample_mode']
                    )
    elif type=='offsets':
        netG = net_nimbusr_offsets(n_iter=opt_net['n_iter'],
            h_nc=opt_net['h_nc'],
            in_nc=opt_net['in_nc'],
            out_nc=opt_net['out_nc'],
            nc=opt_net['nc'],
            nb=opt_net['nb'],
            act_mode=opt_net['act_mode'],
            downsample_mode=opt_net['downsample_mode'],
            upsample_mode=opt_net['upsample_mode']
            )

    if os.path.exists(restoration_network_file):
        print('Loading model for G [{:s}] ...'.format(restoration_network_file))
        netG.load_state_dict(torch.load(restoration_network_file))
    else:
        print('Model does not exists')

    netG = netG.to('cuda')

    return netG


In [ ]:
def restore_images(input_file, reblur_model, net_G, output_folder='testing_results', gamma_factor=1.0,
                   resize_factor=1, network_type='nimbusr_sat', focal_length=-1, save_video=False):

  n_positions= 25 # number of positions estimated by the network
  if not os.path.exists(output_folder):
      os.makedirs(output_folder)

  img_name = input_file.split('/')[-1]
  img_name, ext = img_name.split('.')

  print('loading image ',input_file)
  blurry_image = imread(input_file)/255.0

  blurry_tensor = torch.from_numpy(blurry_image).permute(2,0,1)[None].cuda().float()
  print(blurry_tensor.min(),blurry_tensor.max())
  _, C,H,W = blurry_tensor.shape

  # Kernels and masks are estimated
  blurry_tensor_to_compute_kernels = blurry_tensor**gamma_factor - 0.5


  if focal_length > 0:
      f = torch.Tensor([focal_length]).to(blurry_tensor.device)
      #f = torch.Tensor([float(max(H,W))]).to(tensor_img.device)
      intrinsics = torch.Tensor([[f, 0, W/2],[0, f, H/2], [0, 0, 1] ]).cuda(blurry_tensor.device)
      intrinsics = intrinsics[None,:,:]
  else:
      intrinsics = compute_intrinsics(W,H).cuda(blurry_tensor.device)[None]
      f =  torch.Tensor([max(H,W)]).to(blurry_tensor.device)

      #focal_channel = f/maximo * torch.ones(N,1,H,W).to(tensor_img.device)
      #cam_input = torch.concat((focal_channel, tensor_img), dim=1)

  # get positions
  with torch.no_grad():
      camera_positions = reblur_model(blurry_tensor_to_compute_kernels,f)

  order = sort_positions(camera_positions[0])
  camera_positions[0] = camera_positions[0,order,:]

  offsets = get_offsets_from_positions(blurry_tensor.shape, camera_positions, intrinsics)
  offsets = offsets.reshape(1,2*n_positions, H,W)
  offsets_BT=None

  print(offsets.min(),offsets.max())


  net_G.eval()
  noise_level = 0.01
  noise_level = torch.FloatTensor([noise_level]).view(1,1,1).cuda(blurry_tensor.device)

  print('input_shape', blurry_tensor.shape, offsets.shape)
  with torch.no_grad():
      output = net_G(blurry_tensor, offsets, 1, sigma=noise_level[None,:], offsets_BT=offsets_BT)


  output_img = tensor2im(torch.clamp(output[0].detach(),0,1) - 0.5)
  save_image(output_img, os.path.join(output_folder, img_name + '_PMBM.png' ))

  found_positions_np = camera_positions[0].detach().cpu().numpy()
  np.savetxt(os.path.join(output_folder,f'{img_name}_found_positions.txt'), found_positions_np)
  pose = np.zeros((found_positions_np.shape[0], 6))
  pose[:, 3:] = found_positions_np

  #K, _ = generarK((H,W,C), pose, A=intrinsics[0].detach().cpu().numpy())
  kernels_file = os.path.join(output_folder, img_name + '_kernels_found.png' )
  #kernels_estimated = mostrar_kernels(K, (H,W,C), output_name = kernels_file)


  show_kernels_from_offsets_on_blurry_image(blurry_tensor[0],offsets[0].reshape(n_positions,2,H,W), kernels_file)
  print('Kernels saved in ',os.path.join(output_folder, img_name + '_kernels.png') )

  save_image((255*blurry_image).astype(np.uint8), os.path.join(output_folder, img_name + '.png' ))
  print('Output saved in ', os.path.join(output_folder, img_name + '_J-MTPD.png' ))

  frames, reblurred = generate_video(output, camera_positions, intrinsics[0])
  print(reblurred.shape)
  reblurred = tensor2im(torch.clamp(reblurred[0].detach(),0,1) - 0.5)
  save_image(reblurred, os.path.join(output_folder, img_name + '_reblurred.png' ))

  show_positions_found(found_positions_np, intrinsics[0,0,0].detach().cpu().numpy(), os.path.join(output_folder, img_name + '_positions_found.png'))

  if save_video:
      #imgs=[];
      output_video = os.path.join(output_folder, img_name + '.avi')
      save_video(frames, output_video)
      print('Video saved in ', output_video)


In [ ]:
input_image = 'testing_imgs/0316.png'
input_image = 'testing_imgs/Blurry2_1.png'
restoration_model='pretrained_models/camera_shake_epoch25_epoch35_epoch50_epoch10_epoch5_epoch25_epoch25_epoch25_epoch27_epoch24_epoch4_epoch10_epoch22_epoch23_epoch90_G.pkl'

netG = load_nimbusr_net(restoration_model, 'offsets')

In [ ]:
restore_images(input_image, camera_model, netG)

## Try with your own images

In [ ]:
from google.colab import files
import shutil

upload_folder = 'upload/input'
result_folder = 'upload/output'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(upload_folder)
os.makedirs(result_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

In [ ]:
import glob
input_list = sorted(glob.glob(os.path.join(upload_folder, '*')))
for input_path in input_list:
  img_input = imread(input_path)
  restore_images(input_path, camera_model)